In [67]:
import os
import re
import sys
sys.path.append('../')
import json
import pywt

import numpy as np
import pandas as pd
from scipy.integrate import simpson
from sklearn.linear_model import RidgeClassifier
from tqdm import tqdm
from typing import List
from transform import TimeSeriesTransform
from concurrent.futures import ThreadPoolExecutor, as_completed

In [68]:
cfg = json.load(open("../config/config.json"))

ts_trans = TimeSeriesTransform(cfg)

In [69]:
INSTANCES_DIR = '../data/linear_acuator/instances/'
INFERENCE_DIR = '../data/linear_acuator/inference/'
STATES = ['normal', 
            'backlash1', 'backlash2',
            'lackLubrication1', 'lackLubrication2',
            'spalling1', 'spalling2', 'spalling3', 'spalling4', 'spalling5', 'spalling6', 'spalling7', 'spalling8']
LOADS= ['20kg', '40kg', '-40kg']

In [70]:
load = '20kg'
filenames_20kg = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
filenames_20kg = [filename for sublist in filenames_20kg for filename in sublist]

load = '40kg'
filenames_40kg = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
filenames_40kg = [filename for sublist in filenames_40kg for filename in sublist]

load = '-40kg'
filenames_m40kg = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
filenames_m40kg = [filename for sublist in filenames_m40kg for filename in sublist]

In [71]:
def get_X_y(filenames, load):
    X, y = [], []
    for filename in filenames:
        load_num = load[:-2]
        state = re.match(fr'(.*)_{load_num}', filename).group(1)
        df = pd.read_csv(os.path.join(INFERENCE_DIR, load, state, filename))
        tmp_cur = ts_trans.smoothing(ts_df=df, field='current')
        # tmp_pos = ts_transform.smoothing(ts_df=df, field='position_error')
        X.append(tmp_cur)
        y.append(state)
    return X, y

In [72]:
X_20kg, y_20kg = get_X_y(filenames_20kg, load='20kg')
X_40kg, y_40kg = get_X_y(filenames_40kg, load='40kg')
X_m40kg, y_m40kg = get_X_y(filenames_m40kg, load='-40kg')

In [73]:
lower_bound = 1   # Lower bound of the range (inclusive)
upper_bound = 1000  # Upper bound of the range (exclusive)
n_kernels = 1000
np.random.seed(42)  # for reproducibility
random_scales = np.random.uniform(lower_bound, upper_bound, n_kernels)
wavelet = 'morl'  # Morlet wavelet for CWT

In [74]:
def trapezoidal_integral(x: List[float], y: List[float]) -> float:
    """
    Approximates the integral of a function given its data points using the Trapezoidal Rule.
    
    Parameters:
    x : array-like
        Array of x values (data points along the x-axis, uniformly spaced).
    y : array-like
        Array of y values (function values at each x).
    
    Returns:
    float
        The approximate integral value.
    """
    
    h = x[1:] - x[:-1]  # Step sizes
    integral = np.sum((h / 2) * (y[:-1] + y[1:]))  # Trapezoidal Rule
    return integral

In [75]:
coefficients, _ = pywt.cwt(X_20kg[0], random_scales, wavelet, sampling_period=1/400)
coefficients.shape

(1000, 324)

In [76]:
x_arr = np.arange(0, len(coefficients[0]))
x_arr = np.tile(x_arr, (coefficients.shape[0], 1))
x_arr.shape

(1000, 324)

In [86]:
res = np.array([trapezoidal_integral(x_arr[i], coefficients[i]) for i in range(coefficients.shape[0])])
res.shape

(1000,)

In [78]:
def raven(sample, random_scales, wavelet, trapezoidal_integral):
    coefficients, _ = pywt.cwt(sample, random_scales, wavelet, sampling_period=1/400)
    x_arr = np.arange(0, len(coefficients[0]))
    x_arr = np.tile(x_arr, (coefficients.shape[0], 1))
    integral = np.array([trapezoidal_integral(x_arr[i], coefficients[i]) for i in range(coefficients.shape[0])])
    return integral

In [79]:
def raven_parallel(X, random_scales, wavelet, trapezoidal_integral, max_workers=8):
    raven_results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(raven, X[i], random_scales, wavelet, trapezoidal_integral)
                   for i in range(len(X))]
        for future in tqdm(futures):
            raven_results.append(future.result())
    return raven_results

In [80]:
raven_results_20kg_p = raven_parallel(X_20kg, random_scales, wavelet, trapezoidal_integral)
raven_results_40kg_p = raven_parallel(X_40kg, random_scales, wavelet, trapezoidal_integral)
raven_results_m40kg_p = raven_parallel(X_m40kg, random_scales, wavelet, trapezoidal_integral)

100%|██████████| 195/195 [00:22<00:00,  8.85it/s]


In [81]:
ridge = RidgeClassifier()
ridge.fit(raven_results_20kg_p, y_20kg)
print(f'20kg: {ridge.score(raven_results_20kg_p, y_20kg)}')

20kg: 0.958974358974359


In [82]:
ridge = RidgeClassifier()
ridge.fit(raven_results_40kg_p, y_40kg)
print(f'40kg: {ridge.score(raven_results_40kg_p, y_40kg)}')

40kg: 0.9481865284974094


In [83]:
ridge = RidgeClassifier()
ridge.fit(raven_results_m40kg_p, y_m40kg)
print(f'-40kg: {ridge.score(raven_results_m40kg_p, y_m40kg)}')

-40kg: 0.9435897435897436


In [84]:
ridge.predict(raven_results_m40kg_p)

array(['normal', 'normal', 'normal', 'normal', 'spalling3', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication2',
       'lackLubrication2', 'lackLubrica

In [85]:
np.array(y_m40kg)

array(['normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication2',
       'lackLubrication2', 'lackLubricatio